# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../project.env")
sys.path.append(os.environ["PYTHONPATH"])

In [2]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAIN.architecture.BERT.bert as bert

# Models

## Setup

In [3]:
import torch
import gc

In [ ]:
TRAIN_classes = 'all-classes'
exploringBERT = bert.BERT_MINI
exploring_base_num_epochs = 150
exploring_batch_size = 1024

## Train

In [5]:
sup.bert_score_tracker = []

In [6]:
for data_unit in [sup.DATA_S_PF, sup.DATA_S_PV]:
  bert.find_best(data_unit=data_unit, 
                  label_col=sup.class_numeric_column,
                  class_list=TRAIN_classes, 
                  batch_size=exploring_batch_size,
                  LOADABLE_CANDIDATES=[exploringBERT],
                  )
  
  gc.collect()
  if torch.backends.mps.is_available():
    torch.mps.empty_cache()

gc.collect()
if torch.backends.mps.is_available():
  torch.mps.empty_cache()

{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 15}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 540}


0.7364092705329488
updating best... at 2025-06-21_21:36:05



{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 75}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 540}


0.7230588486596177
not best... at 2025-06-21_21:45:46



{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 15}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 540}


0.9061198333867351
updating best... at 2025-06-21_21:55:30



{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 72}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 540}


0.902168108512229
not best... at 2025-06-21_22:05:03



Data Unit: Spf
Best score: 0.9061198333867351
Best data config: {'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 15}
Best train config: {'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 540}
{'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 15}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 

0.5236875800256082
updating best... at 2025-06-21_22:26:38



{'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 75}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 2160}


0.4788732394366197
not best... at 2025-06-21_22:48:14



{'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 15}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 2160}


0.7708066581306018
updating best... at 2025-06-21_23:09:28



{'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 72}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 2160}


0.8181818181818182
updating best... at 2025-06-21_23:31:12



Data Unit: Spv
Best score: 0.8181818181818182
Best data config: {'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 72}
Best train config: {'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 2160}


# Keep metrics

In [7]:
import pandas as pd
from datetime import datetime

In [8]:
best_scores_df = pd.DataFrame(sup.bert_score_tracker, columns=sup.bert_scores_columns)
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, 
                                          sup.TRAIN_BERT_CODE, exploringBERT))
best_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes,
                                       sup.TRAIN_BERT_CODE, exploringBERT,
                                       f"BERTbest-{now}.csv"), index=False)